In [4]:
!pip install praw
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 1.8 MB/s eta 0:00:00


In [7]:
import os
import re
import pandas as pd
from tqdm import tqdm
from nltk.corpus import stopwords
import praw
import nltk

nltk.download('stopwords')

def limpieza(texto):
    texto = texto.lower()
    texto = re.sub(r'\\[a-z]', ' ', texto)
    texto = re.sub(r'[^a-z]', ' ', texto)
    texto = re.sub(r'\s+[a-z]\s+', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto)
    patern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    texto = patern.sub('', texto)
    texto = texto.strip()
    return texto

def get_reddit_comments(subreddit_name, limit=20):
    reddit = praw.Reddit(client_id='xopGjRv-nvIaf0ba6bM-uQ',
                         client_secret='ieJxpSJ_sonhrB54RU7FbjZoqugZhA',
                         user_agent='Xscrapping')

    comments_list = []
    subreddit = reddit.subreddit(subreddit_name)

    for comment in subreddit.comments(limit=limit):
        comments_list.append({
            'original_text': comment.body,
            'user': comment.author.name if comment.author else 'N/A',
            'date': comment.created_utc
        })

    return comments_list

def scrapping(subreddit_name):
    print(subreddit_name)

    try:
        comments = get_reddit_comments(subreddit_name)

        lista_limpia = []
        for comment in tqdm(comments):
            cleaned_text = limpieza(comment['original_text'])
            comment['cleaned_text'] = cleaned_text
            lista_limpia.append(comment)

        df = pd.DataFrame(lista_limpia)

        print(df)
        condicion = df['cleaned_text'] == ""
        df = df.loc[~condicion]

        # Convertir la fecha a un formato legible
        df['date'] = pd.to_datetime(df['date'], unit='s')
        df['formatted_date'] = df['date'].dt.strftime('%H:%M/%d/%m/%y')

        # Reordenar columnas para mejor legibilidad
        df = df[['original_text', 'cleaned_text', 'user', 'formatted_date']]

        return df

    except Exception as e:
        print(f"Error al obtener comentarios: {e}")
        return pd.DataFrame()  # Retorna un DataFrame vacío en caso de error

# Ejemplo de uso:
if __name__ == '__main__':
    subreddit_name = "depression"
    df = scrapping(subreddit_name)
    print(df.head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



depression


100%|██████████| 20/20 [00:00<00:00, 2666.61it/s]

                                        original_text                 user  \
0   I think one of my main problems is that I cann...     MarzipanMindless   
1                  Friends are mostly bullshit anyway            Shujolnyc   
2   Thank you so much for taking the time you did ...    3verythingsonfire   
3   I felt that title heavy. In my opinion, this l...           MrWheels44   
4   Thank you, everyone. I went to sleep and have ...              Punklin   
5                              How are you feeling? 🧡              Punklin   
6               After dying in fire I agree with you.  Confident-Mine-6378   
7   In my case tbh it's not I've uprooted my whole...            AbaKus420   
8   I hear you. I’ve had body issues my entire lif...       Soft-Fact-4409   
9   I think what I've realised too is that I long ...      VeganBuddhist95   
10  I don't find women who are frail and skinny at...            back2me78   
11  I understand. I'm starting to feel like life i...         su

In [8]:
df

,original_text,cleaned_text,user,formatted_date
0,I think one of my main problems is that I cann...,think one main problems cannot control voice h...,MarzipanMindless,23:39/05/08/24
1,Friends are mostly bullshit anyway,friends mostly bullshit anyway,Shujolnyc,23:37/05/08/24
2,Thank you so much for taking the time you did ...,thank much taking time write kind words sincer...,3verythingsonfire,23:37/05/08/24
3,"I felt that title heavy. In my opinion, this l...",felt title heavy opinion life experience meant...,MrWheels44,23:35/05/08/24
4,"Thank you, everyone. I went to sleep and have ...",thank everyone went sleep woken feeling okay a...,Punklin,23:31/05/08/24
5,How are you feeling? 🧡,feeling,Punklin,23:29/05/08/24
6,After dying in fire I agree with you.,dying fire agree,Confident-Mine-6378,23:27/05/08/24
7,In my case tbh it's not I've uprooted my whole...,case tbh uprooted whole life marriage kids rea...,AbaKus420,23:22/05/08/24
8,I hear you. I’ve had body issues my entire lif...,hear body issues entire life even though run m...,Soft-Fact-4409,23:22/05/08/24
9,I think what I've realised too is that I long ...,think realised long end whether end work contr...,VeganBuddhist95,23:17/05/08/24
